Assignment 2
Segmenting Toronto neighbourhoods

In [19]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [53]:
# getting data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [54]:
# define the dataframe columns
column_names = ['Postalcode', 'Borough', 'Neighborhood'] 
# instantiate the dataframe
data = pd.DataFrame(columns=column_names)
data

,Postalcode,Borough,Neighborhood


In [58]:
# loop through to find postcode, borough, neighborhood 
content = soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()


,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,", ,"
1,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge"
2,M1B\n,Scarborough\n,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
4,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
